# **<center> IMPLEMENTAZIONE </center>**


Scopo di questo progetto è quello di creare un modello per segmentare automaticamente lo stomaco e l'intestino nelle scansioni MRI, basandosi su un data set di queste scansioni per trovare soluzioni creative di deep learning che aiutino i pazienti oncologici a ricevere cure migliori.

Il processo di data mining è una pipeline che si compone delle seguenti fasi:

1. Data collection

2. Data preprocessing

3. Analytical processing

Seguiremo pertanto questa pipeline, suddividendo il notebook in **tre** parti principali e spiegando nel dettaglio le scelte fatte. 

# **<center> DATA COLLECTION </center>**

Questa fase serve a ... ed è di fondamentale importanza perchè buone scelte in questa fase possono avere un impatto significativo sull'intero processo di data mining. 

## Elenco importazioni

In [1]:
import os
import pandas as pd

## Definizione percorsi di base

In [2]:
BASE_DIR = "../BD-Image-Segmentation-Comp/" 
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TRAIN_CSV = os.path.join(BASE_DIR, 'train.csv')

## Caricamento metadati

In questa fase importiamo il modello dal file `train.csv` al fine di ...

In [3]:
# df stands for dataframe
train_df = pd.read_csv(TRAIN_CSV)

# Stampiamo le prime 10 righe del dataframe di partenza
train_df.head(10)

,id,class,segmentation
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN
5,case123_day20_slice_0002,stomach,NaN
6,case123_day20_slice_0003,large_bowel,NaN
7,case123_day20_slice_0003,small_bowel,NaN
8,case123_day20_slice_0003,stomach,NaN
9,case123_day20_slice_0004,large_bowel,NaN


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115488 entries, 0 to 115487
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            115488 non-null  object
 1   class         115488 non-null  object
 2   segmentation  33913 non-null   object
dtypes: object(3)
memory usage: 2.6+ MB


Da una prima lettura, si evince che la maggior parte dei record non presenta alcuna segmentazione.

# **<center> DATA PREPROCESSING </center>**

Finita la parte di data collection, segue la fase di **data preprocessing**, in cui ...

Essa si compone dei seguenti passaggi:

1. Feature extraction
2. Data cleaning
3. Feature selection and trasformation

## FEATURE EXTRACTION

Ci troviamo davanti grandi volumi di dati grezzi ed abbiamo poche indicazioni su come questi dovrebbero essere trasformati in database di feature significative.

### Elenco importazioni

In [5]:
from glob import glob

Il primo passo consiste nell'estrarre gli id dei casi primari. Inoltre, dobbiamo trasformare le maschere di segmentazione in formato `str`, poiché i valori `NaN` saranno difficili da estrarre in altri modi.

Creiamo una funzione ad hoc per modellare ed estrarre alcune informazioni preliminari, chiamata `data_preprocessing` che, ricevuto in input il dataframe non processato, fornisce in output un dataframe in cui vengono estratte:

* la classe per la segmentazione

* il numero del caso clinico

* il giorno di valutazione del caso

* un identificativo per il frame della scansione.

In [6]:
def data_preprocessing(df: pd.DataFrame) -> pd.DataFrame:
    
    df["segmentation"] = df["segmentation"].astype("str")
    
    df["case_id"] = df["id"].apply(lambda x: x.split("_")[0][4:])
    
    df["day_id"] = df["id"].apply(lambda x: x.split("_")[1][3:])
    
    df["slice_id"] = df["id"].apply(lambda x: x.split("_")[-1])
    
    return df

Il dataset appena formattato avrà quindi questa struttura:

In [7]:
train_df = data_preprocessing(train_df)

# Stampiamo le prime 10 righe del dataframe processato
train_df.head(10)

,id,class,segmentation,case_id,day_id,slice_id
0,case123_day20_slice_0001,large_bowel,nan,123,20,0001
1,case123_day20_slice_0001,small_bowel,nan,123,20,0001
2,case123_day20_slice_0001,stomach,nan,123,20,0001
3,case123_day20_slice_0002,large_bowel,nan,123,20,0002
4,case123_day20_slice_0002,small_bowel,nan,123,20,0002
5,case123_day20_slice_0002,stomach,nan,123,20,0002
6,case123_day20_slice_0003,large_bowel,nan,123,20,0003
7,case123_day20_slice_0003,small_bowel,nan,123,20,0003
8,case123_day20_slice_0003,stomach,nan,123,20,0003
9,case123_day20_slice_0004,large_bowel,nan,123,20,0004


Creiamo la funzione `estrai_file_da_id` che, forniti in input il percorso di partenza (da cui costruire la struttura delle cartelle) e l'id del dataframe (da cui ricavare il file), ritorna la stringa del percorso relativo al file ricavato.

In [8]:
def estrai_file_da_id(base_dir: str, case_id: str) -> str:
    
    # Ricaviamo la cartella del caso a partire dall'id salvato nel dataframe, formattato
    # sempre come "caseXYZ_dayDD_slice_SSSS"
    case_folder = case_id.split("_")[0]
    
    # In modo analogo, ricaviamo la cartella del giorno associato al caso appena estratto
    # e l'inizio del nome del file dello slice relativo all'id salvato nel dataframe
    day_folder = "_".join(case_id.split("_")[:2])
    file_starter = "_".join(case_id.split("_")[2:])
    
    # Generiamo, a partire dalle info estratte, il percorso delle scansioni 
    folder = os.path.join(base_dir, case_folder, day_folder, "scans")
    
    # Ricaviamo i file con un nome avente struttura simile (i.e. che iniziano nello stesso
    # modo)
    file = glob(f"{folder}/{file_starter}*")
    
    # Poiché glob genera una lista, siamo forzati a considerare il primo file in modo
    # esplicito, ma la glob restituirà sempre un solo file.
    file = file[0]
    
    return file

Adesso, avendo creato un modo per ottenere gli id delle varie immagini, possiamo procedere all'estrazione dei percorsi.

In [9]:
train_df["path"] = train_df["id"].apply(lambda x: estrai_file_da_id(TRAIN_DIR, x))

# Stampiamo le prime 10 righe del dataframe processato
train_df.head(10)

,id,class,segmentation,case_id,day_id,slice_id,path
0,case123_day20_slice_0001,large_bowel,nan,123,20,0001,../BD-Image-Segmentation-Comp/train/case123/ca...
1,case123_day20_slice_0001,small_bowel,nan,123,20,0001,../BD-Image-Segmentation-Comp/train/case123/ca...
2,case123_day20_slice_0001,stomach,nan,123,20,0001,../BD-Image-Segmentation-Comp/train/case123/ca...
3,case123_day20_slice_0002,large_bowel,nan,123,20,0002,../BD-Image-Segmentation-Comp/train/case123/ca...
4,case123_day20_slice_0002,small_bowel,nan,123,20,0002,../BD-Image-Segmentation-Comp/train/case123/ca...
5,case123_day20_slice_0002,stomach,nan,123,20,0002,../BD-Image-Segmentation-Comp/train/case123/ca...
6,case123_day20_slice_0003,large_bowel,nan,123,20,0003,../BD-Image-Segmentation-Comp/train/case123/ca...
7,case123_day20_slice_0003,small_bowel,nan,123,20,0003,../BD-Image-Segmentation-Comp/train/case123/ca...
8,case123_day20_slice_0003,stomach,nan,123,20,0003,../BD-Image-Segmentation-Comp/train/case123/ca...
9,case123_day20_slice_0004,large_bowel,nan,123,20,0004,../BD-Image-Segmentation-Comp/train/case123/ca...


A partire dai percorsi dei file estratti, possiamo accedere a nuove informazioni, come l'**altezza** e la **larghezza** delle immagini, che andiamo ad usare come ulteriori chiavi del dataframe.

In [11]:
train_df["height"] = train_df["path"].apply(lambda x: os.path.split(x)[-1].split("_")[2]).astype("int")
train_df["width"] = train_df["path"].apply(lambda x: os.path.split(x)[-1].split("_")[3]).astype("int")

train_df.head(10)

,id,class,segmentation,case_id,day_id,slice_id,path,height,width
0,case123_day20_slice_0001,large_bowel,nan,123,20,0001,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
1,case123_day20_slice_0001,small_bowel,nan,123,20,0001,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
2,case123_day20_slice_0001,stomach,nan,123,20,0001,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
3,case123_day20_slice_0002,large_bowel,nan,123,20,0002,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
4,case123_day20_slice_0002,small_bowel,nan,123,20,0002,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
5,case123_day20_slice_0002,stomach,nan,123,20,0002,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
6,case123_day20_slice_0003,large_bowel,nan,123,20,0003,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
7,case123_day20_slice_0003,small_bowel,nan,123,20,0003,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
8,case123_day20_slice_0003,stomach,nan,123,20,0003,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266
9,case123_day20_slice_0004,large_bowel,nan,123,20,0004,../BD-Image-Segmentation-Comp/train/case123/ca...,266,266


## DATA CLEANING

## FEATURE SELECTION AND TRASFORMATION